In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
import random
import math
from torch.utils.tensorboard import SummaryWriter
from matplotlib import pyplot

from pathlib import Path
import requests
import pickle
import gzip

import torch
import math
import torch.nn.functional as F
from torch import nn
from torch import optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE
import copy
from sklearn.metrics import confusion_matrix

pd.options.display.float_format = "{:,.4f}".format
sm = SMOTE(random_state=42)

testset_frac = 0.25
data_samples = 100000 

df_full = pd.read_csv('data/Encoded.csv')
print(df_full.shape)
df_full = df_full.sample(n=data_samples)
df_full = df_full.iloc[:, 1:]
df_full = df_full.drop(columns=['Attack Tool', 'Label', 'sVid', 'dVid', '54'])
df_full = df_full.fillna(df_full.median())

(1215890, 96)


<ipython-input-1-1cad4d3a42e6>:46: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_full = df_full.fillna(df_full.median())


In [2]:
learning_rate = 0.01
numEpoch = 20
batch_size = 32
momentum = 0.9
print_amount=3
number_of_slices = 5
isSmote = False
runtime = 22

file_name = "5G_federated_" + str(isSmote) + "_" + str(number_of_slices)  + "_" + str(runtime) + ".txt"
file = open(file_name, "w")

In [3]:
dummies = pd.get_dummies(df_full['Attack Type'])  # Classification
outcomes = dummies.columns
print(outcomes)
num_classes = len(outcomes)
Y = dummies.values
print(df_full['Attack Type'].value_counts())
print(num_classes)

Index(['Benign', 'HTTPFlood', 'ICMPFlood', 'SYNFlood', 'SYNScan',
       'SlowrateDoS', 'TCPConnectScan', 'UDPFlood', 'UDPScan'],
      dtype='object')
Benign            39272
UDPFlood          37492
HTTPFlood         11588
SlowrateDoS        6076
TCPConnectScan     1676
SYNScan            1653
UDPScan            1289
SYNFlood            855
ICMPFlood            99
Name: Attack Type, dtype: int64
9


In [4]:
df_Benign = df_full[df_full['Attack Type']=='Benign']
df_HTTPFlood= df_full[df_full['Attack Type']=='HTTPFlood']
df_UDPFlood= df_full[df_full['Attack Type']=='UDPFlood']
df_SYNFlood= df_full[df_full['Attack Type']=='SYNScan']
df_SlowrateDoS= df_full[df_full['Attack Type']=='SlowrateDoS']

df_Benign.loc[(df_Benign['Attack Type'] == 'Benign'), 'Attack Type'] = 0
df_HTTPFlood.loc[(df_HTTPFlood['Attack Type'] == 'HTTPFlood'), 'Attack Type'] = 2
df_UDPFlood.loc[(df_UDPFlood['Attack Type'] == 'UDPFlood'), 'Attack Type'] = 1
df_SYNFlood.loc[(df_SYNFlood['Attack Type'] == 'SYNScan'), 'Attack Type'] = 4
df_SlowrateDoS.loc[(df_SlowrateDoS['Attack Type'] == 'SlowrateDoS'), 'Attack Type'] = 3

print(df_Benign.shape)
print(df_HTTPFlood.shape)
print(df_UDPFlood.shape)
print(df_SYNFlood.shape)
print(df_SlowrateDoS.shape)

df_filterd = pd.concat([df_Benign,df_HTTPFlood,df_UDPFlood,df_SYNFlood,df_SlowrateDoS])
print(df_filterd['Attack Type'].value_counts())
print(df_filterd.shape)
type_df = df_filterd['Attack Type'].copy()
data_df = df_filterd.drop('Attack Type',axis=1)
print(type_df.shape)


data_df = data_df / data_df.max()
df_normalized = pd.concat([data_df,type_df],axis=1)

(39272, 90)
(11588, 90)
(37492, 90)
(1653, 90)
(6076, 90)
0    39272
1    37492
2    11588
3     6076
4     1653
Name: Attack Type, dtype: int64
(96081, 90)
(96081,)


In [5]:
def divide_train_test(df, propotion=0.1):
    
    df_train = {}
    df_test = []
    for key,val in df['Attack Type'].value_counts().iteritems():
        df_part = df[df['Attack Type'] == key]
        df_part = df_part.dropna(axis=1)
        df_test.append(df_part[0: int(df_part.shape[0]*propotion)])
        df_train[key] = df_part[int(df_part.shape[0]*propotion):df_part.shape[0]]
        
    return df_train,df_test

In [6]:
def get_data_for_slices(df_train, number_of_slices, isSmote=False, x_name="x_train", y_name="y_train"):
    
    df_normal = df_train.get(0) # benign
    df_dos = df_train.get(1) # dos = udpflood
    df_u2r = df_train.get(2) # u2r = httpflood
    df_r2u = df_train.get(3) # r2u = slowratedos
    df_probe = df_train.get(4) # probe = synscan
    
    df_normal_s1 = df_normal[int(df_normal.shape[0]*0/number_of_slices):int(df_normal.shape[0]*(1)/number_of_slices)]
    df_normal_s2 = df_normal[int(df_normal.shape[0]*1/number_of_slices):int(df_normal.shape[0]*(2)/number_of_slices)]
    df_normal_s3 = df_normal[int(df_normal.shape[0]*2/number_of_slices):int(df_normal.shape[0]*(3)/number_of_slices)]
    df_normal_s4 = df_normal[int(df_normal.shape[0]*3/number_of_slices):int(df_normal.shape[0]*(4)/number_of_slices)]
    df_normal_s5 = df_normal[int(df_normal.shape[0]*4/number_of_slices):int(df_normal.shape[0]*(5)/number_of_slices)]

    df_s1 = pd.concat([df_normal_s1,df_dos])
    df_s2 = pd.concat([df_normal_s2,df_u2r])
    df_s3 = pd.concat([df_normal_s3,df_r2u])
    df_s4 = pd.concat([df_normal_s4,df_probe])
    df_s5 = df_normal_s5

    x_data_dict= dict()
    y_data_dict= dict()

    y_s1 = (df_s1.pop('Attack Type').values).astype('int')
    unique, counts = np.unique(y_s1, return_counts=True)
    print('slice 1: ')
    print(dict(zip(unique, counts)))
    y_info1 = torch.tensor(y_s1).type(torch.LongTensor)
    x_info1 = torch.tensor(df_s1.values).float()

    y_s2 = (df_s2.pop('Attack Type').values).astype('int')
    unique, counts = np.unique(y_s2, return_counts=True)
    print('slice 2: ')
    print(dict(zip(unique, counts)))
    y_info2 = torch.tensor(y_s2).type(torch.LongTensor)
    x_info2 = torch.tensor(df_s2.values).float()

    y_s3 = (df_s3.pop('Attack Type').values).astype('int')
    unique, counts = np.unique(y_s3, return_counts=True)
    print('slice 3: ')
    print( dict(zip(unique, counts)))
    y_info3 = torch.tensor(y_s3).type(torch.LongTensor)
    x_info3 = torch.tensor(df_s3.values).float()

    y_s4 = (df_s4.pop('Attack Type').values).astype('int')
    unique, counts = np.unique(y_s4, return_counts=True)
    print('slice 4: ')
    print(dict(zip(unique, counts)))
    y_info4 = torch.tensor(y_s4).type(torch.LongTensor)
    x_info4 = torch.tensor(df_s4.values).float()

    y_s5 = (df_s5.pop('Attack Type').values).astype('int')
    unique, counts = np.unique(y_s5, return_counts=True)
    print('slice 5: ')
    print(dict(zip(unique, counts)))
    y_info5 = torch.tensor(y_s5).type(torch.LongTensor)
    x_info5 = torch.tensor(df_s5.values).float()


    x_data_dict.update({"x_train0" : x_info1})
    y_data_dict.update({"y_train0" : y_info1})

    x_data_dict.update({"x_train1" : x_info2})
    y_data_dict.update({"y_train1" : y_info2})

    x_data_dict.update({"x_train2" : x_info3})
    y_data_dict.update({"y_train2" : y_info3})

    x_data_dict.update({"x_train3" : x_info4})
    y_data_dict.update({"y_train3" : y_info4})

    x_data_dict.update({"x_train4" : x_info5})
    y_data_dict.update({"y_train4" : y_info5})
    
    return x_data_dict, y_data_dict 

In [7]:
df_train, df_test = divide_train_test(df_normalized,propotion=0.1)
print(len(df_train))
# print('Value counts in train set : ')
# df_train['Attack Type'].value_counts()
# print('Value counts in test set : ')
# print(df_test['Attack Type'].value_counts())

x_train_dict, y_train_dict = get_data_for_slices(df_train, number_of_slices, isSmote)

df_test = pd.concat(df_test)
y_test = df_test.pop('Attack Type').values
x_test = df_test.values

print('Test set size is : x => ' + str(x_test.shape) + ' y => ' + str(y_test.shape))
x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test.astype('int')).type(torch.LongTensor)

inputs = x_test.shape[1]
outputs = 5
print(inputs,outputs)

5
slice 1: 
{0: 7069, 1: 33743}
slice 2: 
{0: 7069, 2: 10430}
slice 3: 
{0: 7069, 3: 5469}
slice 4: 
{0: 7069, 4: 1488}
slice 5: 
{0: 7069}
Test set size is : x => (9606, 85) y => (9606,)
85 5


In [8]:
class Net2nn(nn.Module):
    def __init__(self, inputs, outputs):
        super(Net2nn, self).__init__()
        self.fc1=nn.Linear(inputs,200)
        self.fc2=nn.Linear(200,200)
        self.fc3=nn.Linear(200,outputs)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [9]:
class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

In [10]:
def train(model, train_loader, criterion, optimizer):
    model.train()
    train_loss = 0.0
    correct = 0

    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        prediction = output.argmax(dim=1, keepdim=True)
        correct += prediction.eq(target.view_as(prediction)).sum().item()
        

    return train_loss / len(train_loader), correct/len(train_loader.dataset)

In [11]:
def validation(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            
            test_loss += criterion(output, target).item()
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    test_loss /= len(test_loader)
    correct /= len(test_loader.dataset)

    return (test_loss, correct)

In [12]:
def confusion_mat(model, test_loader):
    y_pred = []
    y_true = []

    # iterate over test data
    for inputs, labels in test_loader:
        output = model(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # Save Prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
    precisionv = precision_score(y_true,y_pred,average='macro')
    recallv = recall_score(y_true,y_pred,average='macro')
    print('precision value: '+str(precisionv))
    print('recall value: '+ str(recallv))
#     df_cm = pd.DataFrame(cf_matrix, index=[i for i in Counter(y_test)],
#                          columns=[i for i in Counter(y_test)])
#     plt.figure(1)
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')
#     plt.figure(figsize=(12, 7))

#     sn.heatmap(df_cm, annot=True).set(xlabel='Predicted label', ylabel='True label')
#     plt.savefig('D:\\learning\\PyTorch\\experiment\\cf\\cf_fl_'+str(self.number_of_slices)+'.png')
    print('confusion matrix for normal scenario for slices : ' + str(number_of_slices))
    print(cf_matrix)
    file.write('\ncf matrix for slice :' + str(number_of_slices))
    file.write('\n'+str(cf_matrix))

In [13]:
def create_model_optimizer_criterion_dict(number_of_slices):
    model_dict = dict()
    optimizer_dict= dict()
    criterion_dict = dict()
    
    for i in range(number_of_slices):
        model_name="model"+str(i)
        model_info=Net2nn(inputs, outputs)
        model_dict.update({model_name : model_info })
        
        optimizer_name="optimizer"+str(i)
        optimizer_info = torch.optim.SGD(model_info.parameters(), lr=learning_rate, momentum=momentum)
        optimizer_dict.update({optimizer_name : optimizer_info })
        
        criterion_name = "criterion"+str(i)
        criterion_info = nn.CrossEntropyLoss()
        criterion_dict.update({criterion_name : criterion_info})
        
    return model_dict, optimizer_dict, criterion_dict 

In [14]:
def get_averaged_weights(model_dict, number_of_slices):
   
    fc1_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc1.weight.shape)
    fc1_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc1.bias.shape)
    
    fc2_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc2.weight.shape)
    fc2_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc2.bias.shape)
    
    fc3_mean_weight = torch.zeros(size=model_dict[name_of_models[0]].fc3.weight.shape)
    fc3_mean_bias = torch.zeros(size=model_dict[name_of_models[0]].fc3.bias.shape)
    
    with torch.no_grad():
    
    
        for i in range(number_of_slices):
            fc1_mean_weight += model_dict[name_of_models[i]].fc1.weight.data.clone()
            fc1_mean_bias += model_dict[name_of_models[i]].fc1.bias.data.clone()
        
            fc2_mean_weight += model_dict[name_of_models[i]].fc2.weight.data.clone()
            fc2_mean_bias += model_dict[name_of_models[i]].fc2.bias.data.clone()
        
            fc3_mean_weight += model_dict[name_of_models[i]].fc3.weight.data.clone()
            fc3_mean_bias += model_dict[name_of_models[i]].fc3.bias.data.clone()

        
        fc1_mean_weight =fc1_mean_weight/number_of_slices
        fc1_mean_bias = fc1_mean_bias/ number_of_slices
    
        fc2_mean_weight =fc2_mean_weight/number_of_slices
        fc2_mean_bias = fc2_mean_bias/ number_of_slices
    
        fc3_mean_weight =fc3_mean_weight/number_of_slices
        fc3_mean_bias = fc3_mean_bias/ number_of_slices
    
    return fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias

In [15]:
def set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices):
    fc1_mean_weight, fc1_mean_bias, fc2_mean_weight, fc2_mean_bias, fc3_mean_weight, fc3_mean_bias = get_averaged_weights(model_dict, number_of_slices=number_of_slices)
    with torch.no_grad():
        main_model.fc1.weight.data = fc1_mean_weight.data.clone()
        main_model.fc2.weight.data = fc2_mean_weight.data.clone()
        main_model.fc3.weight.data = fc3_mean_weight.data.clone()

        main_model.fc1.bias.data = fc1_mean_bias.data.clone()
        main_model.fc2.bias.data = fc2_mean_bias.data.clone()
        main_model.fc3.bias.data = fc3_mean_bias.data.clone() 
    return main_model

In [16]:
def compare_local_and_merged_model_performance(number_of_slices):
    accuracy_table=pd.DataFrame(data=np.zeros((number_of_slices,3)), columns=["sample", "local_ind_model", "merged_main_model"])
    for i in range (number_of_slices):
    
        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size=batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        individual_loss, individual_accuracy = validation(model, test_dl, criterion)
        main_loss, main_accuracy =validation(main_model, test_dl, main_criterion )
    
        accuracy_table.loc[i, "sample"]="sample "+str(i)
        accuracy_table.loc[i, "local_ind_model"] = individual_accuracy
        accuracy_table.loc[i, "merged_main_model"] = main_accuracy

    return accuracy_table

In [17]:
def send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices):
    with torch.no_grad():
        for i in range(number_of_slices):
            print('Updating model :' + name_of_models[i] )
            model_dict[name_of_models[i]].fc1.weight.data =main_model.fc1.weight.data.clone()
            model_dict[name_of_models[i]].fc2.weight.data =main_model.fc2.weight.data.clone()
            model_dict[name_of_models[i]].fc3.weight.data =main_model.fc3.weight.data.clone() 
            
            model_dict[name_of_models[i]].fc1.bias.data =main_model.fc1.bias.data.clone()
            model_dict[name_of_models[i]].fc2.bias.data =main_model.fc2.bias.data.clone()
            model_dict[name_of_models[i]].fc3.bias.data =main_model.fc3.bias.data.clone() 
    
    return model_dict

In [18]:
def start_train_end_node_process(number_of_slices):
    for i in range (number_of_slices): 

        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

#         valid_ds = TensorDataset(x_valid_dict[name_of_x_valid_sets[i]], y_valid_dict[name_of_y_valid_sets[i]])
#         valid_dl = DataLoader(valid_ds, batch_size=batch_size * 2)
        
        test_ds = TensorDataset(x_test_dict[name_of_x_test_sets[i]], y_test_dict[name_of_y_test_sets[i]])
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        print("Subset" ,i)
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
#             valid_loss, valid_accuracy = validation(model, valid_dl, criterion)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
    
            print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))

In [19]:

def start_train_end_node_process_without_print(number_of_slices):
    for i in range (number_of_slices): 

        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        for epoch in range(numEpoch):        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)

In [20]:
def start_train_end_node_process_print_some(number_of_slices, print_amount):
    for i in range (number_of_slices): 
        
        print('Federated learning for slice '+ str(i+1))
        train_ds = TensorDataset(x_train_dict[name_of_x_train_sets[i]], 
                                 y_train_dict[name_of_y_train_sets[i]])
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

        test_ds = TensorDataset(x_test, y_test)
        test_dl = DataLoader(test_ds, batch_size= batch_size * 2)
    
        model=model_dict[name_of_models[i]]
        criterion=criterion_dict[name_of_criterions[i]]
        optimizer=optimizer_dict[name_of_optimizers[i]]
    
        if i<print_amount:
            print("Subset" ,i)
            
        for epoch in range(numEpoch):
        
            train_loss, train_accuracy = train(model, train_dl, criterion, optimizer)
            test_loss, test_accuracy = validation(model, test_dl, criterion)
            
            if i<print_amount:        
                print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.5f}".format(train_accuracy) + " | test accuracy: {:7.5f}".format(test_accuracy))    

In [21]:
# initial_model = Net2nn()
# initial_optimizer = torch.optim.SGD(initial_model.parameters(), lr=0.01, momentum=0.9)
# initial_criterion = nn.CrossEntropyLoss()

centralized_model = Net2nn(inputs, outputs)
centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
centralized_criterion = nn.CrossEntropyLoss()

In [22]:
print("------ Centralized Model ------")

train_acc = []
test_acc = []
train_loss = []
test_loss = []

test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(test_ds, batch_size=batch_size * 2)

for i in range(number_of_slices):
    centralized_model = Net2nn(inputs,outputs)
    centralized_optimizer = torch.optim.SGD(centralized_model.parameters(), lr=0.01, momentum=0.9)
    centralized_criterion = nn.CrossEntropyLoss()
#     centralized_model = copy.deepcopy(initial_model)
#     centralized_optimizer = copy.deepcopy(initial_optimizer)
#     centralized_criterion = copy.deepcopy(initial_criterion)
    print('Training with slice ' + str(i+1) + ' data' )
    x_name = 'x_train' + str(i)
    y_name = 'y_train' + str(i)
    train_ds = TensorDataset(x_train_dict[x_name], y_train_dict[y_name])
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    for epoch in range(numEpoch):
        central_train_loss, central_train_accuracy = train(centralized_model, train_dl, centralized_criterion, centralized_optimizer)
        central_test_loss, central_test_accuracy = validation(centralized_model, test_dl, centralized_criterion)
        
        train_acc.append(central_train_accuracy)
        train_loss.append(central_train_loss)
        test_acc.append(central_test_accuracy)
        test_loss.append(central_test_loss)
        
        print("epoch: {:3.0f}".format(epoch+1) + " | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    print(" | train accuracy: {:7.4f}".format(central_train_accuracy) + " | test accuracy: {:7.4f}".format(central_test_accuracy))
    confusion_mat(centralized_model, test_dl)
    
print("------ Training finished ------")
print('Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
print('Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))


------ Centralized Model ------
Training with slice 1 data
epoch:   1 | train accuracy:  0.9614 | test accuracy:  0.7778
epoch:   2 | train accuracy:  0.9943 | test accuracy:  0.7844
epoch:   3 | train accuracy:  0.9946 | test accuracy:  0.7934
epoch:   4 | train accuracy:  0.9954 | test accuracy:  0.7941
epoch:   5 | train accuracy:  0.9957 | test accuracy:  0.7921
epoch:   6 | train accuracy:  0.9957 | test accuracy:  0.7953
epoch:   7 | train accuracy:  0.9958 | test accuracy:  0.7934
epoch:   8 | train accuracy:  0.9962 | test accuracy:  0.7937
epoch:   9 | train accuracy:  0.9956 | test accuracy:  0.7929
epoch:  10 | train accuracy:  0.9969 | test accuracy:  0.7933
epoch:  11 | train accuracy:  0.9968 | test accuracy:  0.7945
epoch:  12 | train accuracy:  0.9971 | test accuracy:  0.7944
epoch:  13 | train accuracy:  0.9966 | test accuracy:  0.7962
epoch:  14 | train accuracy:  0.9971 | test accuracy:  0.7949
epoch:  15 | train accuracy:  0.9970 | test accuracy:  0.7965
epoch:  16 

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.3283814574391221
recall value: 0.39837524282003595
confusion matrix for normal scenario for slices : 5
[[3915   12    0    0    0]
 [  19 3730    0    0    0]
 [1158    0    0    0    0]
 [ 607    0    0    0    0]
 [   2  163    0    0    0]]
Training with slice 2 data
epoch:   1 | train accuracy:  0.9656 | test accuracy:  0.5219
epoch:   2 | train accuracy:  0.9951 | test accuracy:  0.5229
epoch:   3 | train accuracy:  0.9955 | test accuracy:  0.5232
epoch:   4 | train accuracy:  0.9963 | test accuracy:  0.5236
epoch:   5 | train accuracy:  0.9966 | test accuracy:  0.5239
epoch:   6 | train accuracy:  0.9974 | test accuracy:  0.5254
epoch:   7 | train accuracy:  0.9979 | test accuracy:  0.5269
epoch:   8 | train accuracy:  0.9985 | test accuracy:  0.5263
epoch:   9 | train accuracy:  0.9985 | test accuracy:  0.5255
epoch:  10 | train accuracy:  0.9987 | test accuracy:  0.5275
epoch:  11 | train accuracy:  0.9987 | test accuracy:  0.5276
epoch:  12 | train accuracy:

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.24504172781329497
recall value: 0.39923605805958745
confusion matrix for normal scenario for slices : 5
[[3912    0   15    0    0]
 [3749    0    0    0    0]
 [   0    0 1158    0    0]
 [ 212    0  395    0    0]
 [ 165    0    0    0    0]]
Training with slice 3 data
epoch:   1 | train accuracy:  0.9576 | test accuracy:  0.4659
epoch:   2 | train accuracy:  0.9930 | test accuracy:  0.4660
epoch:   3 | train accuracy:  0.9947 | test accuracy:  0.4676
epoch:   4 | train accuracy:  0.9957 | test accuracy:  0.4681
epoch:   5 | train accuracy:  0.9965 | test accuracy:  0.4687
epoch:   6 | train accuracy:  0.9969 | test accuracy:  0.4691
epoch:   7 | train accuracy:  0.9970 | test accuracy:  0.4689
epoch:   8 | train accuracy:  0.9972 | test accuracy:  0.4696
epoch:   9 | train accuracy:  0.9972 | test accuracy:  0.4688
epoch:  10 | train accuracy:  0.9978 | test accuracy:  0.4688
epoch:  11 | train accuracy:  0.9974 | test accuracy:  0.4704
epoch:  12 | train accuracy

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.18555192017162633
recall value: 0.3984003785728759
confusion matrix for normal scenario for slices : 5
[[3915    0    0   12    0]
 [3749    0    0    0    0]
 [ 446    0    0  712    0]
 [   3    0    0  604    0]
 [ 165    0    0    0    0]]
Training with slice 4 data
epoch:   1 | train accuracy:  0.9500 | test accuracy:  0.4256
epoch:   2 | train accuracy:  0.9991 | test accuracy:  0.4258
epoch:   3 | train accuracy:  0.9991 | test accuracy:  0.4259
epoch:   4 | train accuracy:  0.9995 | test accuracy:  0.4259
epoch:   5 | train accuracy:  0.9995 | test accuracy:  0.4259
epoch:   6 | train accuracy:  0.9995 | test accuracy:  0.4259
epoch:   7 | train accuracy:  0.9996 | test accuracy:  0.4260
epoch:   8 | train accuracy:  0.9996 | test accuracy:  0.4260
epoch:   9 | train accuracy:  0.9996 | test accuracy:  0.4260
epoch:  10 | train accuracy:  0.9996 | test accuracy:  0.4259
epoch:  11 | train accuracy:  0.9996 | test accuracy:  0.4259
epoch:  12 | train accuracy:

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.28319034000635523
recall value: 0.4
confusion matrix for normal scenario for slices : 5
[[3927    0    0    0    0]
 [3749    0    0    0    0]
 [1158    0    0    0    0]
 [ 607    0    0    0    0]
 [   0    0    0    0  165]]
Training with slice 5 data
epoch:   1 | train accuracy:  0.9873 | test accuracy:  0.4088
epoch:   2 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   3 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   4 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   5 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   6 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   7 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   8 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:   9 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:  10 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:  11 | train accuracy:  1.0000 | test accuracy:  0.4088
epoch:  12 | train accuracy:  1.0000 | test

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
file.write('\nCentralized Mean train accuracy: ' + str(sum(train_acc)/len(train_acc)))
file.write('\nCentralized Mean test accuracy: ' + str(sum(test_acc)/len(test_acc)))

51

In [24]:
print(x_train_dict["x_train1"].shape, y_train_dict["y_train1"].shape)
print(x_test.shape, y_test.shape)

torch.Size([17499, 85]) torch.Size([17499])
torch.Size([9606, 85]) torch.Size([9606])


In [25]:
main_model = Net2nn(inputs,outputs)
main_optimizer = torch.optim.SGD(main_model.parameters(), lr=learning_rate, momentum=0.9)
main_criterion = nn.CrossEntropyLoss()

In [26]:
model_dict, optimizer_dict, criterion_dict = create_model_optimizer_criterion_dict(number_of_slices)

In [27]:
name_of_x_train_sets=list(x_train_dict.keys())
name_of_y_train_sets=list(y_train_dict.keys())

name_of_models=list(model_dict.keys())
name_of_optimizers=list(optimizer_dict.keys())
name_of_criterions=list(criterion_dict.keys())

print(name_of_x_train_sets)
print(name_of_y_train_sets)
print("\n ------------")
print(name_of_models)
print(name_of_optimizers)
print(name_of_criterions)

['x_train0', 'x_train1', 'x_train2', 'x_train3', 'x_train4']
['y_train0', 'y_train1', 'y_train2', 'y_train3', 'y_train4']

 ------------
['model0', 'model1', 'model2', 'model3', 'model4']
['optimizer0', 'optimizer1', 'optimizer2', 'optimizer3', 'optimizer4']
['criterion0', 'criterion1', 'criterion2', 'criterion3', 'criterion4']


In [28]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0700,  0.0199,  0.0060,  0.0049, -0.0007]],
       grad_fn=<SliceBackward>)
tensor([[ 0.0313, -0.0698, -0.0490,  0.0331, -0.0566]],
       grad_fn=<SliceBackward>)


In [29]:
model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4


In [30]:
print(main_model.fc2.weight[0:1,0:5])
print(model_dict["model1"].fc2.weight[0:1,0:5])

tensor([[-0.0700,  0.0199,  0.0060,  0.0049, -0.0007]],
       grad_fn=<SliceBackward>)
tensor([[-0.0700,  0.0199,  0.0060,  0.0049, -0.0007]],
       grad_fn=<SliceBackward>)


In [31]:
# start_train_end_node_process()
start_train_end_node_process_print_some(number_of_slices, print_amount)

Federated learning for slice 1
Subset 0
epoch:   1 | train accuracy: 0.96099 | test accuracy: 0.78441
epoch:   2 | train accuracy: 0.99373 | test accuracy: 0.79471
epoch:   3 | train accuracy: 0.99471 | test accuracy: 0.78940
epoch:   4 | train accuracy: 0.99476 | test accuracy: 0.77358
epoch:   5 | train accuracy: 0.99473 | test accuracy: 0.79315
epoch:   6 | train accuracy: 0.99581 | test accuracy: 0.79065
epoch:   7 | train accuracy: 0.99468 | test accuracy: 0.79211
epoch:   8 | train accuracy: 0.99613 | test accuracy: 0.79138
epoch:   9 | train accuracy: 0.99571 | test accuracy: 0.79263
epoch:  10 | train accuracy: 0.99625 | test accuracy: 0.79398
epoch:  11 | train accuracy: 0.99598 | test accuracy: 0.79513
epoch:  12 | train accuracy: 0.99674 | test accuracy: 0.79315
epoch:  13 | train accuracy: 0.99686 | test accuracy: 0.79305
epoch:  14 | train accuracy: 0.99667 | test accuracy: 0.79502
epoch:  15 | train accuracy: 0.99713 | test accuracy: 0.79450
epoch:  16 | train accuracy: 0

In [32]:
## As you can see, wieghts of local models are updated after training process
print(main_model.fc2.weight[0,0:5])
print(model_dict["model1"].fc2.weight[0,0:5])

tensor([-0.0700,  0.0199,  0.0060,  0.0049, -0.0007], grad_fn=<SliceBackward>)
tensor([-0.0713,  0.0199,  0.0057,  0.0047, -0.0008], grad_fn=<SliceBackward>)


In [33]:
before_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
before_test_loss, before_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nbefore training main model')
confusion_mat(main_model, test_dl)

main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 

after_acc_table=compare_local_and_merged_model_performance(number_of_slices=number_of_slices)
after_test_loss, after_test_accuracy = validation(main_model, test_dl, main_criterion)
file.write('\nafter training main model')
confusion_mat(main_model, test_dl)

D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.009179062636000861
recall value: 0.085948880076218
confusion matrix for normal scenario for slices : 5
[[  12   42    0 3872    1]
 [   0    0    0 3749    0]
 [ 615   24    0  518    1]
 [ 139  209    0  259    0]
 [   1    0    0  164    0]]
precision value: 0.08176139912554653
recall value: 0.2
confusion matrix for normal scenario for slices : 5
[[3927    0    0    0    0]
 [3749    0    0    0    0]
 [1158    0    0    0    0]
 [ 607    0    0    0    0]
 [ 165    0    0    0    0]]


D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
print("Federated main model vs individual local models before FedAvg first iteration")
file.write('\nBefore training federated')
file.write('\n'+str(before_acc_table))
before_acc_table

Federated main model vs individual local models before FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.7956,0.0282
1,sample 1,0.5279,0.0282
2,sample 2,0.4699,0.0282
3,sample 3,0.4260,0.0282
4,sample 4,0.4088,0.0282


In [35]:
print("Federated main model vs individual local models after FedAvg first iteration")
file.write('\nAfter training federated')
file.write('\n'+str(after_acc_table))
after_acc_table

Federated main model vs individual local models after FedAvg first iteration


,sample,local_ind_model,merged_main_model
0,sample 0,0.7956,0.4088
1,sample 1,0.5279,0.4088
2,sample 2,0.4699,0.4088
3,sample 3,0.4260,0.4088
4,sample 4,0.4088,0.4088


In [36]:
print("Before 1st iteration main model accuracy on all test data: {:7.4f}".format(before_test_accuracy))
print("After 1st iteration main model accuracy on all test data: {:7.4f}".format(after_test_accuracy))
print("Centralized model accuracy on all test data: {:7.4f}".format(central_test_accuracy))

Before 1st iteration main model accuracy on all test data:  0.0282
After 1st iteration main model accuracy on all test data:  0.4088
Centralized model accuracy on all test data:  0.4088


In [37]:
for i in range(100):
    model_dict=send_main_model_to_nodes_and_update_model_dict(main_model, model_dict, number_of_slices)
    start_train_end_node_process_without_print(number_of_slices)
    main_model= set_averaged_weights_as_main_model_weights_and_update_main_model(main_model,model_dict, number_of_slices) 
    test_loss, test_accuracy = validation(main_model, test_dl, main_criterion)
    print("Iteration", str(i+2), ": main_model accuracy on all test data: {:7.4f}".format(test_accuracy)) 
    confusion_mat(main_model, test_dl)

Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 2 : main_model accuracy on all test data:  0.4695


D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.2140338554334079
recall value: 0.3010561926136446
confusion matrix for normal scenario for slices : 5
[[3924    0    3    0    0]
 [3749    0    0    0    0]
 [ 572    0  586    0    0]
 [ 246    0  361    0    0]
 [ 165    0    0    0    0]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 3 : main_model accuracy on all test data:  0.4088


D:\Softwares\anaconda-python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


precision value: 0.08176139912554653
recall value: 0.2
confusion matrix for normal scenario for slices : 5
[[3927    0    0    0    0]
 [3749    0    0    0    0]
 [1158    0    0    0    0]
 [ 607    0    0    0    0]
 [ 165    0    0    0    0]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 4 : main_model accuracy on all test data:  0.4329


...

precision value: 0.6345331334083901
recall value: 0.40906200682650606
confusion matrix for normal scenario for slices : 5
[[3927    0    0    0    0]
 [2344 1405    0    0    0]
 [ 802    0  311   45    0]
 [ 327    0   36  244    0]
 [  13  152    0    0    0]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 8 : main_model accuracy on all test data:  0.5937
precision value: 0.8403758869504518
recall value: 0.4665793998927318
confusion matrix for normal scenario for slices : 5
[[3927    0    0    0    0]
 [2603 1146    0    0    0]
 [ 796    0  312   50    0]
 [ 319    0   23  265    0]
 [   7  105    0    0   53]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 9 : main_model accuracy on all test data:  0.6279
precision value: 0.8600919890031327
recall value: 0.5169856146197824
confusion matrix for normal scenario for slices : 5
[[39

Iteration 26 : main_model accuracy on all test data:  0.8475
precision value: 0.921094246721837
recall value: 0.8624593272407793
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [1096 2653    0    0    0]
 [ 206    0  883   69    0]
 [  79    0   13  515    0]
 [   1    0    0    0  164]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 27 : main_model accuracy on all test data:  0.9179
precision value: 0.9391624950622217
recall value: 0.9091403508282161
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 484 3265    0    0    0]
 [ 141    0  928   89    0]
 [  63    0   10  534    0]
 [   1    0    0    0  164]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 28 : main_model accuracy on all test data:  0.8571
precision value: 0.9244261718233389
recall value: 0.869417859579

Iteration 45 : main_model accuracy on all test data:  0.9594
precision value: 0.9624259658478435
recall value: 0.934639381887628
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 140 3609    0    0    0]
 [ 109    0  998   51    0]
 [  65    0   23  519    0]
 [   1    0    0    0  164]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 46 : main_model accuracy on all test data:  0.8825
precision value: 0.9349201662824214
recall value: 0.9029831184792491
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 927 2822    0    0    0]
 [  73    0 1033   52    0]
 [  50    0   25  532    0]
 [   1    0    0    0  164]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 47 : main_model accuracy on all test data:  0.9502
precision value: 0.9599790314246393
recall value: 0.940684781381

Iteration 64 : main_model accuracy on all test data:  0.9276
precision value: 0.9452861227868354
recall value: 0.9400114983520769
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 548 3201    0    0    0]
 [  29    0 1047   82    0]
 [  19    0   16  572    0]
 [   0    0    0    0  165]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 65 : main_model accuracy on all test data:  0.8726
precision value: 0.9280677875864963
recall value: 0.911477084016526
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [1077 2672    0    0    0]
 [  32    0 1049   77    0]
 [  20    0   17  570    0]
 [   0    0    0    0  165]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 66 : main_model accuracy on all test data:  0.9778
precision value: 0.9604537454232874
recall value: 0.965270733837

Iteration 83 : main_model accuracy on all test data:  0.9356
precision value: 0.9495234120977788
recall value: 0.9485358400652644
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 495 3254    0    0    0]
 [  20    0 1059   79    0]
 [   9    0   15  583    0]
 [   0    0    0    0  165]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 84 : main_model accuracy on all test data:  0.9805
precision value: 0.9711640240825588
recall value: 0.9669766494501367
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [  52 3697    0    0    0]
 [  26    0 1069   63    0]
 [  28    0   17  562    0]
 [   0    0    0    0  165]]
Updating model :model0
Updating model :model1
Updating model :model2
Updating model :model3
Updating model :model4
Iteration 85 : main_model accuracy on all test data:  0.9324
precision value: 0.9509231509967225
recall value: 0.94896460149

In [38]:
confusion_mat(main_model, test_dl)

precision value: 0.9522511221289168
recall value: 0.948923231798051
confusion matrix for normal scenario for slices : 5
[[3926    0    0    1    0]
 [ 614 3135    0    0    0]
 [   8    0 1098   52    0]
 [   4    0   20  583    0]
 [   0    0    0    0  165]]


In [39]:
file.close()